In [1]:
from src.dataset import DataModuleFakeReviews
from src.model import FakeReviewsLightning
import torch
from pytorch_lightning.callbacks import ModelCheckpoint, ModelSummary, EarlyStopping
from pytorch_lightning import Trainer
from lightning.pytorch.tuner import Tuner
from clearml import Task

c:\Users\mmapa\anaconda3\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# set Task credentials
Task.set_credentials(
    #  api_host="https://api.clear.ml", 
    #  web_host="https://app.clear.ml", 
    #  files_host="https://files.clear.ml", 
    #  key='6ZHX9UQMYL874A1NE8', 
    #  secret='=2h6#%@Y&m*tC!VLEXq&JI7QhZPKuJfbaYD4!uUk(t7=9ENv'
)

In [ ]:
task = Task.init(project_name="Fake Review Detection",
                    task_name="finetune RoBERTa - v1.0")

# initialize data module and model
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
print(device)
print("   \   ")
print("   ^ ^ ")
print("  {'.'}")
print("? /  | ")
print("\|  || ")

# enable cudnn benchmarking if available
torch.backends.cudnn.benchmark = True if torch.backends.cudnn.is_available() and cuda_available else False

torch.set_float32_matmul_precision('high')

datamodule = DataModuleFakeReviews(batch_size=4, num_workers=8)

model = FakeReviewsLightning(clearml_logger=task.get_logger(), device=device).to(device)

# initialize checkpoint callback depending on parse arguments
checkpoint_callback = ModelCheckpoint(dirpath="checkpoints/", monitor="val_loss", mode="min")

trainer = Trainer(accelerator="gpu", max_epochs=1, profiler="simple",
                        callbacks=[checkpoint_callback, ModelSummary(4),
                                    EarlyStopping(monitor="val_loss",
                                                mode="min",
                                                patience=10)],
                        strategy="auto", enable_checkpointing=True, limit_train_batches=10, limit_val_batches=10)
# tuner = Tuner(trainer)
# tuner.scale_batch_size(model, datamodule=datamodule, mode="power")

# else:
#     print("No GPU")

# fit model
trainer.fit(model=model, datamodule=datamodule)